# SQL-03. Соединение таблиц.

## Описание таблиц о футбольных матчах и командах.

Вы уже умеете делать запросы к одной таблице, использовать агрегатные функции и фильтровать данные в выводе. Но в реальных условиях базы данных обычно содержат множество таблиц и при запросе необходимо обращаться к нескольким таблицам. Освоением этого навыка мы сейчас и займёмся.

В этом модуле мы будем работать с таблицами о футбольных матчах и командах.

Таблицы этого модуля, как и все другие в курсе, лежат в схеме `sql` в [Metabase](http://sql.skillfactory.ru:3000/). Нам понадобятся таблицы `teams` и `matches`.


**Таблица** `teams` **с данными о командах**

| Название столбца | Содержимое столбца           |
|------------------|------------------------------|
| id               | id команды                   |
| api_id           | ключ на таблицу matches      |
| long_name        | полное название команды      |
| short_name       | сокращённое название команды |



**Таблица** `matches` **с данными о матчах**

| Название столбца | Содержимое столбца                             |
|------------------|------------------------------------------------|
| id               | id матча                                       |
| season           | сезон                                          |
| date             | дата матча                                     |
| home_team_api_id | api_id домашней команды, ключ на таблицу teams |
| away_team_api_id | api_id гостевой команды, ключ на таблицу teams |
| home_team_goals  | количество голов домашней команды              |
| away_team_goals  | количество голов гостевой команды              |

**Задание  1.1**

Сколько различных полных названий команд в таблице `teams`?

```sql
SELECT
    COUNT(DISTINCT long_name)
FROM
    sql.teams
```

**Ответ:** 296

Сколько в таблице teams команд с коротким названием *VAL*?

```sql
SELECT
    COUNT(*)
FROM
    sql.teams
WHERE
    short_name = 'VAL'
```

**Ответ:** 3

Информацию о скольких матчах содержит таблица `matches`?

```sql
SELECT
    COUNT(*)
FROM
    sql.matches
```

**Ответ:** 25083



Данные за какие сезоны даны в таблице matches?
Ответ введите в формате 2019/2020.

```sql
SELECT
    DISTINCT season
FROM
    sql.matches
ORDER BY
    season
```

**Ответ:** 2008/2009 - 2015/2016

**Заданике 1.2**

Напишите запрос, который выведет сезон (`season`), а также общее количество забитых мячей домашними (`total_home_goals`) и гостевыми (`total_away_goals`) командами.

Отсортируйте по столбцу с сезоном в порядке возрастания.

```sql
SELECT
    season,
    SUM(home_team_goals) AS total_home_goals,
    SUM(away_team_goals) AS total_away_goals
FROM
    sql.matches
GROUP BY
    season
ORDER BY
    season ASC
```

## Объединение таблиц без операторов

Чтобы соединить две таблицы между собой, достаточно записать названия таблиц через запятую в разделе `FROM`. Что произойдёт в таком случае?

```sql
SELECT *
FROM
    sql.teams,
    sql.matches
```

Каждая запись, которая есть в таблице `teams`, будет соединена с каждой записью в таблице `matches`.

Это действие также называют декартовым произведением таблиц.

Действительно ли это произведение?

Легко проверить! В исходных таблицах `teams` и `matches` было `299` и `25083` записей соответственно. Если соединить каждую запись одной таблицы с каждой записью другой, получится `299` `*` `25083` записей в итоговой таблице.

**Задание 2.1**

Напишите запрос, который выведет количество строк соединённой таблицы.

```sql
SELECT
    COUNT(*)
FROM
    sql.matches,
    sql.teams
```

В данном случае соединение таблиц не даёт практической пользы: мы получили очень много записей, которые никак не можем интерпретировать, потому что команды не соответствуют матчам.

Давайте исправим это. В таблице teams есть столбец `api_id`, а таблица `matches` содержит столбцы `home_team_api_id` и `away_team_api_id` — это ключи таблиц, по которым они соединяются.

**Ключ** — это поле (столбец) в таблице, которое позволяет однозначно идентифицировать запись (строку).

Чтобы соединить таблицы и получить данные о домашней команде по каждому матчу, добавим условие `where home_team_api_id = api_id`.

```SQL
SELECT
    *
FROM
    sql.teams,
    sql.matches
WHERE
    home_team_api_id = api_id
```

Аналогично можем получить данные о гостевых командах: необходимо изменить условие на
where `away_team_api_id = api_id`.

```SQL
SELECT
    *
FROM
    sql.teams,
    sql.matches
WHERE
    away_team_api_id = api_id
```

Итак, мы только что объединили таблицы по ключу.

Вы уже знакомы с ключами по таблице `pokemon` (там в этой роли выступал столбец `id`). Ключи нужны для того, чтобы иметь возможность не перепутать между собой различные записи.

Например, у нас есть несколько команд с одинаковым названием: *Polonia* *Bytom*, *Widzew* *Łódź* и *Royal* *Excel* *Mouscron* — хотя это разные команды, с разными `id`.

Кроме того, как мы уже смогли убедиться, ключи используются для соединения таблиц между собой.

**Ключи бывают двух основных типов:**

* *Primary* — первичный ключ — служит для идентификации текущей таблицы и, как правило, идёт первым в списке столбцов. Всегда уникален: повторяющихся значений в основной таблице быть не может.<br><br>
* *Foreign* — внешний ключ — представляет собой ссылку на другую таблицу.<br><br>

Как правило, названия ключей имеют «хвост», который позволяет их идентифицировать: например, `_id`, `_rk`, `_cd`, `_pk` (от `primary_key`), `_fk` (от `foreign_key`) и другие.

**Обратите внимание!** В данном датасете ключ `api_id` таблицы `teams` может быть использован в разных значениях. Его можно использовать для того, чтобы получить информацию о домашней (`home`) или гостевой (`away`) команде.

Вы могли заметить, что в последних двух запросах получилось очень много столбцов. Как и при работе с одиночной таблицей, мы можем выбирать, какие столбцы соединённой таблицы выводить.

С помощью известного нам запроса получим названия команд, игравших домашние матчи, и счёт матчей.

```sql
SELECT
    long_name,
    home_team_goals,
    away_team_goals
FROM
    sql.teams,
    sql.matches
WHERE
    home_team_api_id = api_id
```

**Задание 2.2**

Напишите запрос, который выведет таблицу с результатами матчей для гостевых команд, содержащую:

* названия гостевых команд (`long_name`),<br><br>
* количество забитых мячей домашней команды (`home_team_goals`),<br><br>
* количество забитых мячей гостевой команды (`away_team_goals`).<br><br>

```sql
SELECT
    long_name,
    home_team_goals,
    away_team_goals
FROM
    sql.teams,
    sql.matches
WHERE
    away_team_api_id = api_id
```

## JOIN

В прошлом юните для соединения таблиц мы использовали условие в разделе `WHERE`, чтобы показать принцип работы оператора `JOIN`.

В качестве примера используем запрос из предыдущего юнита.

```SQL
SELECT
    long_name,
    home_team_goals,
    away_team_goals
FROM
    sql.teams,
    sql.matches
WHERE
    home_team_api_id = api_id
```

и запишем его с использованием `JOIN`.

```sql
SELECT
    long_name,
    home_team_goals,
    away_team_goals
FROM
    sql.teams
    JOIN sql.matches ON home_team_api_id = api_id
```

**Задание 3.1**

Выполните данный запрос

```sql
SELECT
    *
FROM
    sql.teams,
    sql.matches
WHERE
    away_team_api_id = api_id
```

с использованием оператора `JOIN`, исключив оператор `WHERE`.

```sql
SELECT
    *
FROM
    sql.teams
    JOIN sql.matches ON away_team_api_id = api_id
```

**СИНТАКСИС**

Оператор `JOIN` упрощает процесс соединения таблиц.

Его синтаксис можно представить следующим образом:

```sql
SELECT
    столбец1,
    столбец2,
    ...
FROM
    таблица1
    JOIN таблица2 ON условие
```

Порядок присоединения таблиц в данном случае не важен — результат будет одинаковым.

С помощью `JOIN` можно соединить и более двух таблиц.

```sql
SELECT
    столбец1,
    столбец2,
    ...
FROM
    таблица1
    JOIN таблица2 ON условие
    JOIN таблица3 ON условие
```

В таблицах, которые мы соединяем, могут быть одинаковые названия столбцов.

К примеру, столбец `id` есть и в таблице `matches`, и в таблице `teams`. Такой запрос не будет обработан.

```sql
SELECT
    id
FROM
    sql.teams
    JOIN sql.matches ON home_team_api_id = api_id
```

В результате должно появиться сообщение об ошибке вроде такого: *"... column id is ambiguous ..."*.

Что же делать в таком случае?

Можно указать, откуда мы хотим запросить данные, записав название таблицы перед столбцом через точку.

```sql
SELECT
    teams.id
FROM
    sql.teams
    JOIN sql.matches ON home_team_api_id = api_id
```

Можно также выбрать и столбец из таблицы `matches`.

```sql
SELECT
    matches.id
FROM
    sql.teams
    JOIN sql.matches on home_team_api_id = api_id
```

Зачастую названия таблиц слишком длинные, так что использовать их неудобно.

Упростить обращение к различным таблицам можно, присвоив им сокращённые названия — **алиасы** (от англ. *alias*).

Синтаксис для указания алиаса такой же, как и для названия столбца.

```sql
SELECT
    столбец1,
    столбец2,
    ...
FROM
    таблица1 AS короткое_название_1
    JOIN таблица2 AS короткое_название_2 ON условие
```

Название записывается без пробелов и операторов.

Если необходимо записать название, в котором используются пробелы ("table 1"), то алиас можно обернуть в кавычки.

**Важно!** Обращаться по такому алиасу придётся также с помощью кавычек.

```sql
SELECT
    "table 1".столбец1,
    "table 2".столбец2,
    ...
FROM
    таблица1 AS "table 1"
    JOIN таблица2 AS "table 2" ON условие
```

**Использование таких алиасов считается плохой практикой как минимум по причине того, что обращаться с такими алиасами неудобно.**

Вместо алиасов с кавычками рекомендуется давать простое короткое название на латинице, без специальных символов и пробелов. Если таблиц немного и все названия начинаются с разных букв, можно присваивать алиасы по первой букве.

Ключевое слово `as`, как и в названии столбца, можно опустить в большинстве СУБД.

```sql
SELECT
    столбец1 новое_название_столбца,
    столбец2 новое_название_столбца,
    ...
FROM
    таблица1 короткое_название_1
    JOIN таблица2 короткое_название_2 ON условие
```

**Задание 3.2**

Напишите запрос, который выведет два столбца: id матча (`match_id`) и id домашней команды (`team_id`). Отсортируйте по id матча в порядке возрастания значений.

```sql
SELECT
    matches.id AS match_id,
    teams.id AS team_id
FROM
    sql.matches
    JOIN sql.teams ON home_team_api_id = api_id
ORDER BY
    match_id ASC
```

Кажется, теперь мы можем соединить таблицы между собой и создать своё табло со счётом матчей, как на спортивных сайтах!

Давайте с помощью запроса SQL получим таблицу, содержащую:

* название домашней команды;<br><br>
* количество забитых домашней командой голов;<br><br>
* количество забитых гостевой командой голов;<br><br>
* название гостевой команды.<br><br>

```sql
SELECT
    h.long_name "домашняя команда",
    m.home_team_goals "голы домашней команды",
    m.away_team_goals "голы гостевой команды",
    a.long_name "гостевая команда"
FROM
    sql.matches m
    JOIN sql.teams h ON m.home_team_api_id = h.api_id
    JOIN sql.teams a ON m.away_team_api_id = a.api_id
```

**Задание 3.3**

Напишите запрос, который выведет столбцы:

* id матча,<br><br>
* короткое название домашней команды (home_short),<br><br>
* короткое название гостевой команды (away_short).<br><br>

Отсортируйте запрос по возрастанию id матча.

```sql
SELECT
    m.id,
    h.short_name AS home_short,
    a.short_name AS away_short
FROM
    sql.matches AS m
    JOIN sql.teams AS h ON m.home_team_api_id = h.api_id
    JOIN sql.teams AS a ON m.away_team_api_id = a.api_id
ORDER BY
    m.id
```

## Фильтрация и агрегатные функции

Соединять таблицы мы научились, теперь давайте научимся получать необходимые данные из нескольких таблиц.

Принцип построения запроса и порядок операторов такой же, как и с обычной таблицей.

Вспомним его:

```sql
SELECT... 
FROM... 
WHERE... 
GROUP BY... 
ORDER BY... 
LIMIT...
```

**Фильтрация данных**

К соединённым таблицам применимы функции фильтрации данных.

Например, можно вывести *id* матчей, в которых команда *Arsenal* была гостевой.

```sql
SELECT
    m.id
FROM
    sql.teams t
    JOIN sql.matches m ON m.away_team_api_id = t.api_id
WHERE
    long_name = 'Arsenal'
```

Принципиальное отличие фильтрации данных по соединённым таблицам от аналогичного действия по одиночным таблицам заключается в том, что, фильтруя записи одной таблицы, мы также будем фильтровать и записи другой таблицы, поскольку соединённые на уровне запроса таблицы по сути являются единой таблицей.

Например, результат запроса

```sql
SELECT
    m.id id_1,
    m.season,
    t.id id_2,
    t.long_name
FROM
    sql.teams t
    JOIN sql.matches m ON m.away_team_api_id = t.api_id
```

можно разделить на две разные части


![](data/dst3-u2-md3_4_1.png)

Одна часть — таблица `matches` с алиасом `m`, вторая — `teams` с алиасом `t`, но после соединения они являются одной таблицей.

Таким образом, если вы отфильтруете данные по одной части таблицы, то другая, соединённая, часть пропадёт вместе с ней.

**Задание 4.1**

Напишите запрос, который выведет полное название домашней команды (`long_name`), количество голов домашней команды (`home_goal`) и количество голов гостевой команды (`away_goal`) в матчах, где домашней командой были команды с коротким названием 'GEN'. Отсортируйте запрос по id матча в порядке возрастания.

```sql
SELECT
    t.long_name,
    m.home_team_goals AS home_goal,
    m.away_team_goals AS away_goal
FROM
    sql.teams AS t
    JOIN sql.matches AS m ON t.api_id = m.home_team_api_id
WHERE
    t.short_name = 'GEN'
ORDER BY
    m.id ASC
```